# IAPI Controller Demo

This notebook to demonstrate how we can use pythonnet (http://pythonnet.github.io/) to call Thermo's Instrument Application Programming Interface
 (https://github.com/thermofisherlsms/iapi) libraries from Python and uses a controller to handle the scans.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import glob
import datetime
import time
from pathlib import Path

In [24]:
import sys
sys.path.append('C:\\Users\\joewa\\Work\\git\\vimms')
sys.path.append('C:\\Users\\Vinny\\work\\vimms')

from vimms.MassSpec import IndependentMassSpectrometer, IAPIMassSpectrometer
from vimms.Controller import IdleController, SimpleMs1Controller, TopNController, RoiController, HybridController
from vimms.Environment import Environment, IAPIEnvironment
from vimms.Common import *

In [4]:
ref_dir = 'C:\\Users\\joewa\\Work\\git\\vimms\\Interface\\FusionLibrary\\bin\\Debug\\netstandard2.0'

In [5]:
ref_dir = 'C:\\Users\\joewa\\Work\\git\\vimms\\Interface\\FusionLibrary\\bin\\Debug\\netstandard2.0'
if ref_dir not in sys.path:
    sys.path.append(ref_dir)

In [6]:
filename = 'C:\\Users\\joewa\\University of Glasgow\\Vinny Davies - CLDS Metabolomics Project\\Data\\multibeers_urine_data\\beers\\fragmentation\\mzML\\Beer_multibeers_1_T10_POS.mzML'
show_console_logs = True

In [7]:
# filename = None
# show_console_logs = False

## Test with Top-N Controller

### Verbose mode

In [8]:
# ionisation_mode = POSITIVE
# N = 10
# isolation_width = 1
# mz_tol = 10
# rt_tol = 15
# min_ms1_intensity = 1.75E5
# controller = TopNController(ionisation_mode, N, isolation_width, mz_tol, rt_tol, min_ms1_intensity)

In [9]:
# mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

In [10]:
# pbar = False
# set_log_level_debug()
# max_time = 5

In [11]:
# env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test.mzML')
# env.run()

### Normal mode

In [12]:
ionisation_mode = POSITIVE
N = 10
isolation_width = 1
mz_tol = 10
rt_tol = 15
min_ms1_intensity = 1.75E5
controller = TopNController(ionisation_mode, N, isolation_width, mz_tol, rt_tol, min_ms1_intensity)

In [13]:
mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

2019-12-16 13:05:32.267 | DEBUG    | vimms.MassSpec:__init__:633 - AddReference: FusionLibrary, Version=1.0.0.0, Culture=neutral, PublicKeyToken=null
2019-12-16 13:05:32.272 | DEBUG    | vimms.MassSpec:__init__:636 - ListAssemblies: ['mscorlib', 'clrmodule', 'Python.Runtime', 'System.Core', 'System.Configuration', 'System.Xml', 'System', '__CodeGenerator_Assembly', 'e__NativeCall_Assembly', 'FusionLibrary', 'netstandard', 'Fusion.API-1.0', 'API-2.0', 'Spectrum-1.0']


In [14]:
pbar = True
set_log_level_warning()
max_time = 100

In [15]:
env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test_TopN.mzML')
env.run()

(99.991s) ms_level=2 N=10 DEW=15: 100%|███████████████████████████████████████████████████████████████████▉| 99.99086952209473/100 [01:39<00:00,  1.00it/s]2019-12-16 13:07:13.180 | WARNING  | vimms.MassSpec:close:708 - Closing fusion bridge
(99.991s) ms_level=2 N=10 DEW=15: 100%|███████████████████████████████████████████████████████████████████▉| 99.99086952209473/100 [01:40<00:00,  1.00s/it]


## Test with ROI Controller

In [16]:
ionisation_mode = POSITIVE
N = 10
isolation_width = 1
mz_tol = 10
rt_tol = 15
min_ms1_intensity = 1.75E5
min_roi_intensity = 5000
min_roi_length = 10
score_method = 'Top N'

controller = RoiController(ionisation_mode, isolation_width, mz_tol, min_ms1_intensity,
                           min_roi_intensity, min_roi_length, score_method, N, rt_tol)

In [17]:
mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

In [18]:
pbar = True
set_log_level_warning()
max_time = 100

In [19]:
env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test_ROI.mzML')
env.run()

(99.819s) ms_level=2 N=10 DEW=15: 100%|███████████████████████████████████████████████████████████████████▉| 99.81894946098328/100 [01:39<00:00,  1.00s/it]c:\users\joewa\.virtualenvs\vimms-ewhzmunz\lib\site-packages\tqdm\std.py:484: TqdmWarning: clamping frac to range [0, 1]
  charset=Bar.ASCII if ascii is True else ascii or Bar.UTF)
(100.023s) ms_level=1 N=10 DEW=15: 100%|██████████████████████████████████████████████████████████████████| 100.02339935302734/100 [01:40<00:00,  1.00s/it]2019-12-16 13:09:02.223 | WARNING  | vimms.MassSpec:close:708 - Closing fusion bridge
(100.023s) ms_level=1 N=10 DEW=15: 100%|██████████████████████████████████████████████████████████████████| 100.02339935302734/100 [01:40<00:00,  1.00s/it]


## Test with Full Purity Controller

In [20]:
ionisation_mode = POSITIVE
isolation_window = [1]   # the isolation window in Dalton around a selected precursor ion
N = [5]
rt_tol = [30]
mz_tol = [10]
min_ms1_intensity = 1.75E5
scan_param_changepoints = None
# rt_range = [(0, 1440)]
rt_range = [(0, 100)]

In [21]:
n_purity_scans=N[0]
purity_shift=0.2
purity_threshold=1

In [22]:
mass_spec = IAPIMassSpectrometer(ionisation_mode, ref_dir, filename=filename, show_console_logs=show_console_logs)

In [25]:
hybrid_controller = HybridController(mass_spec, N, scan_param_changepoints, isolation_window, mz_tol, rt_tol, min_ms1_intensity, n_purity_scans, purity_shift, purity_threshold)

In [26]:
pbar = True
set_log_level_warning()
max_time = 100

In [27]:
env = IAPIEnvironment(mass_spec, controller, max_time, progress_bar=pbar, out_dir='results', out_file='iapi_test_purity.mzML')
env.run()

(99.999s) ms_level=2 N=10 DEW=15: 100%|███████████████████████████████████████████████████████████████████▉| 99.99908685684204/100 [01:40<00:00,  1.00it/s]2019-12-16 13:11:26.831 | WARNING  | vimms.MassSpec:close:708 - Closing fusion bridge
(99.999s) ms_level=2 N=10 DEW=15: 100%|███████████████████████████████████████████████████████████████████▉| 99.99908685684204/100 [01:40<00:00,  1.00s/it]
c:\users\joewa\.virtualenvs\vimms-ewhzmunz\lib\site-packages\psims\document.py:92: ReferentialIntegrityWarning: Overwriting existing value for 100000 with 'SPECTRUM_100000' in store Spectrum
  key, value, self.type_name), ReferentialIntegrityWarning)
c:\users\joewa\.virtualenvs\vimms-ewhzmunz\lib\site-packages\psims\document.py:92: ReferentialIntegrityWarning: Overwriting existing value for 100001 with 'SPECTRUM_100001' in store Spectrum
  key, value, self.type_name), ReferentialIntegrityWarning)
c:\users\joewa\.virtualenvs\vimms-ewhzmunz\lib\site-packages\psims\document.py:92: ReferentialIntegri